In [77]:
import panel as pn
pn.extension('tabulator', sizing_mode='stretch_width', notifications=True)
from panel.io.notifications import NotificationArea
NotificationArea.position = 'bottom-right'

import pyvo as vo
import datetime as dt
from astropy.time import Time
import param
from pypika import Table, Criterion, EmptyCriterion, Order
import pandas as pd
from astroquery.simbad import Simbad
from astropy.coordinates import SkyCoord
from astropy import units as u
import warnings
from astroquery.exceptions import TableParseError
from astropy.io import fits
from matplotlib.figure import Figure
from matplotlib import cm
import numpy as np
from pathlib import Path

NotificationArea(sizing_mode='stretch_width')

In [78]:
url = "https://vo.astro.rug.nl/tap"
service = vo.dal.TAPService(url)

In [79]:
from pypika.queries import Query, QueryBuilder
from pypika.utils import builder, QueryException
from typing import Any, Optional
from enum import Enum

class Dialects(Enum):
    ADQL = 'adql'

class ADQLQuery(Query):
    @classmethod
    def _builder(cls, **kwargs: Any) -> "QueryBuilder":
        return ADQLQueryBuilder(**kwargs)

class ADQLQueryBuilder(QueryBuilder):
    QUOTE_CHAR = None
    ALIAS_QUOTE_CHAR = '"'
    QUERY_ALIAS_QUOTE_CHAR = ''
    QUERY_CLS = ADQLQuery

    def __init__(self, **kwargs: Any) -> None:
        super().__init__(dialect=Dialects.ADQL, **kwargs)
        self._contains = None
        self._top = None

    @builder
    def top(self, value):
        try:
            self._top = int(value)
        except ValueError:
            raise QueryException('TOP value must be an integer')

    def get_sql(self, *args, **kwargs):
        return super(ADQLQueryBuilder, self).get_sql(*args, groupby_alias=False, **kwargs)

    def _top_sql(self):
        if self._top:
            return 'TOP {} '.format(self._top)
        else:
            return ''

    @builder
    def contains(self, from_table, ra, dec, cone_radius):
        self._contains = [from_table, ra, dec, cone_radius]

    def _contains_sql(self):
        if self._contains:
            return "CONTAINS(POINT('ICRS', {}.ra, {}.dec), CIRCLE('ICRS', {}, {}, {}))=1 AND ".format(
                self._contains[0], self._contains[0], self._contains[1], self._contains[2], self._contains[3])
        else:
            return ''

    def _select_sql(self, **kwargs: Any) -> str:
        return "SELECT {distinct}{top}{select}".format(
            top=self._top_sql(),
            distinct="DISTINCT " if self._distinct else "",
            select=",".join(term.get_sql(with_alias=True, subquery=True, **kwargs) for term in self._selects)
        )

    @builder
    def _where_sql(self, quote_char: Optional[str] = None, **kwargs: Any) -> str:
        return " WHERE {contains}{where}".format(
            contains=self._contains_sql(),
            where=self._wheres.get_sql(quote_char=quote_char, subquery=True, **kwargs)
        )

In [80]:
options = {'Filename': 'filename',
           'Observation date': 'kw_DATE_OBS',
           'Image type': 'kw_IMAGETYP',
           'Right ascension': 'ra',
           'Declination': 'dec',
           'Object': 'kw_OBJECT',
           'Filter': 'kw_FILTER'}
options_reverse = {'filename': 'Filename',
                   'kw_DATE_OBS': 'Observation date',
                   'kw_IMAGETYP': 'Image type',
                   'ra': 'Right ascension',
                   'dec': 'Declination',
                   'kw_OBJECT': 'Object',
                   'kw_FILTER': 'Filter'}
class CompileQuery(param.Parameterized):
    begin_date = dt.datetime(2020, 4, 22, 12)
    end_date = dt.datetime(2020, 4, 23, 12)
    dates = param.DateRange((begin_date, end_date))
    dates_order = param.Selector({'Descending': Order.desc, 'Ascending': Order.asc})
    reset_date_button = param.Action(lambda x: x.param.trigger('reset_date_button'), label='Reset date')
    select_columns = param.ListSelector(default=['Filename',
                                                 'Image type',
                                                 'Filter',
                                                 'Observation date',
                                                 'Right ascension',
                                                 'Declination',
                                                 'Object'],
                                        objects=list(options.keys()))
    ra_dec_not_null = param.Boolean(default=True)
    select_object = param.String(default='')
    cone_radius = param.Number(default=0.1, bounds=(0.0, 3.0))

    @param.depends('reset_date_button', watch=True)
    def reset_date(self):
        self.dates = (dt.datetime(2008, 5, 1, 12), dt.datetime(2022, 5, 1, 12))

    def table_string(self):
        table_name = 'observations.raw'
        return table_name

    def from_query(self):
        raw_observations = Table(self.table_string())
        return raw_observations

    @param.depends('select_columns', watch=True)
    def select_query(self):
        select_options = ''
        for i in self.select_columns:
            select_options += str(self.from_query()).strip('"') + '.' + options[i] + ','
        return select_options.rstrip(',')

    @param.depends('select_object', watch=True)
    def find_object(self):
        if self.select_object == '':
            return True
        else:
            object_name = self.select_object
            with warnings.catch_warnings():
                warnings.filterwarnings('error')
                try:
                    simbad_object = Simbad.query_object(object_name)
                    coordinates = SkyCoord(simbad_object['RA'], simbad_object['DEC'], unit=(u.hourangle, u.deg))
                    object_ra, object_dec = coordinates.ra.deg[0], coordinates.dec.deg[0]
                    return object_ra, object_dec
                except (Warning, TableParseError):
                    return False

    @param.depends('ra_dec_not_null', watch=True)
    def not_null_check(self):
        if self.ra_dec_not_null:
            return True

    @param.depends('dates', watch=True)
    def date_query(self):
        start_date = Time(self.dates[0]).jd
        end_date = Time(self.dates[1]).jd
        return start_date, end_date

In [81]:
class FetchData(CompileQuery):
    search_object_button = param.Action(lambda x: x.param.trigger('search_object_button'), label='Search')
    reset_search_button = param.Action(lambda x: x.param.trigger('reset_search_button'), label='Reset search')

    @param.depends('reset_search_button', watch=True)
    def reset_search(self):
        self.dates = (self.begin_date, self.end_date)
        self.ra_dec_not_null = True
        self.select_object = ''
        self.cone_radius = 0.1
        self.param.trigger('search_object_button')

    @param.depends('search_object_button')
    def data(self):
        if not self.find_object():
            return
        observation_table = self.from_query()
        dates = self.date_query()
        query = ADQLQuery.from_(
            observation_table
        ).select(
            self.select_query()
        ).where(
            Criterion.all([
                observation_table.ra.isnotnull() if self.not_null_check() else EmptyCriterion(),
                observation_table.dec.isnotnull() if self.not_null_check() else EmptyCriterion(),
                observation_table.obs_jd[dates[0]:dates[1]]
            ])
        ).orderby(observation_table.obs_jd, order=self.dates_order).top(500)
        if self.select_object != '':
            query = query.contains(self.table_string(), self.find_object()[0], self.find_object()[1], self.cone_radius)
        result = service.search(query.get_sql())
        data_pandas = result.to_table().to_pandas()
        return data_pandas

    @param.depends('search_object_button')
    def display_object_error(self):
        if not self.find_object():
            message = "SIMBAD could not find the object " + str(self.select_object) + " that you searched for."
            pn.state.notifications.error(message, duration=0)

    @param.depends('search_object_button', 'data')
    def display_data_error(self):
        if self.find_object() and self.data().values.size == 0:
            message = "No entries could be found with the options you specified."
            pn.state.notifications.error(message, duration=0)

In [128]:
class CreateTable(FetchData):
    table = param.DataFrame(columns=CompileQuery().select_columns, precedence=-1)
    if FetchData().data() is None:
        table.default = pd.DataFrame(columns=FetchData().select_columns)
    else:
        table.default = FetchData().data()
        table.columns = options_reverse

    def __init__(self, **params):
        super().__init__(**params)
        self.table_widget = pn.widgets.Tabulator(self.table,
                                                 disabled=True,
                                                 pagination='local',
                                                 show_index=False,
                                                 layout='fit_columns',
                                                 selectable='checkbox-single',
                                                 text_align='left',
                                                 min_height=500,
                                                 titles=options_reverse,
                                                 selection=[2, 4])

    @param.depends('data', watch=True)
    def full_table(self):
        if self.data() is None:
            self.table_widget.value = pd.DataFrame(columns=self.select_columns)
        else:
            self.table_widget.value = self.data()
            self.table_widget.titles = options_reverse

In [141]:
class CreatePlots(CreateTable):
    def __init__(self, **params):
        super().__init__(**params)
        self.selected_table_widget = pn.widgets.Tabulator(disabled=True,
                                                          pagination='local',
                                                          show_index=False,
                                                          layout='fit_columns',
                                                          selectable='checkbox-single',
                                                          text_align='left',
                                                          min_height=500,
                                                          titles=options_reverse)

    @param.depends('table_widget.selected_dataframe')
    def selected_table(self):
        if self.table_widget.selected_dataframe is None:
            self.selected_table_widget.value = pd.DataFrame()
            return self.selected_table_widget
        else:
            self.selected_table_widget.value = self.table_widget.selected_dataframe[['filename', 'kw_IMAGETYP', 'kw_FILTER']]
            self.selected_table_widget.value['filename'] = [self.selected_table_widget.value['filename'].replace(i, Path(i).name) for i in self.selected_table_widget.value['filename']]
            # for i in self.selected_table_widget.value['filename']:
            #     filename = Path(i).name
            #     self.selected_table_widget.value['filename'] = self.selected_table_widget.value['filename'].replace(i, filename)
            return self.selected_table_widget

    fits_file = fits.open('../Data/Raw/200422_Li_.00000040.NGC_5907.FIT')
    fits_data = fits_file[0].data
    vmin = np.percentile(fits_data, 5)
    vmax = np.percentile(fits_data, 95)

    def create_plot(self):
        fig0 = Figure(figsize=(8, 6))
        ax0 = fig0.subplots()
        img = ax0.imshow(self.fits_data, interpolation='none', origin='lower', cmap=cm.gray, vmin=self.vmin, vmax=self.vmax)
        fig0.colorbar(img)
        return pn.pane.Matplotlib(fig0, tight=True, sizing_mode='scale_both')

In [142]:
test = CreatePlots()
print(test.selected_table().value)

                                            filename  kw_IMAGETYP kw_FILTER
2  2                     200422_Li_.00000118.NGC_...  Light Frame    O[III]
4  2    /net/vega/data/users/observatory/images/2...  Light Frame     S[II]


In [85]:
class CreateView(CreatePlots):
    def panel(self):
        widgets = {
            'dates': pn.widgets.DatetimeRangePicker,
            'dates_order': pn.widgets.Select,
            'reset_date_button': {'widget_type': pn.widgets.Button, 'button_type': 'warning'},
            'select_columns': {'widget_type': pn.widgets.CrossSelector, 'definition_order': False},
            'ra_dec_not_null': pn.widgets.Checkbox,
            'cone_radius': {'widget_type': pn.widgets.FloatSlider, 'name': 'Cone radius (degrees)' , 'step': 0.01},
            'search_object_button': {'widget_type': pn.widgets.Button, 'button_type': 'primary'},
            'reset_search_button': {'widget_type': pn.widgets.Button, 'button_type': 'warning'}
        }
        settings = pn.Row(
            pn.Param(self, widgets=widgets, width=385, sizing_mode="fixed", name="Settings")
        )
        # settings = pn.Tabs(
        #     pn.Row(
        #         pn.Param(self, widgets=widgets, width=385, sizing_mode="fixed", name="Settings")
        #     ),
        #     pn.Row(
        #         pn.Param(self, widgets=widgets, width=385, sizing_mode="fixed", name="Settings2")
        #     )
        # )
        bootstrap.sidebar.append(settings)
        tabs = pn.Tabs(
            ('Data Table', pn.Column(self.table_widget, self.display_object_error, self.display_data_error)),
            ('Data Plotting', pn.Row(pn.Card(self.selected_table), pn.Card(self.create_plot)))
        )
        bootstrap.main.append(tabs)
        return bootstrap

In [86]:
bootstrap = pn.template.BootstrapTemplate(title='Bootstrap Template', sidebar_width=400)

test = CreateView()
test.panel().servable();
# print(test.selected_table().value)

                           filename  kw_IMAGETYP kw_FILTER
2  200422_Li_.00000118.NGC_6543.FIT  Light Frame    O[III]
4  200422_Li_.00000117.NGC_6543.FIT  Light Frame     S[II]


In [1]:
# import panel as pn
#
# pn.extension()
#
# selector = pn.widgets.Select(
#     value=pn.widgets.ColorPicker, options=[
#         pn.widgets.ColorPicker,
#         pn.widgets.DatePicker,
#         pn.widgets.FileInput,
#         pn.widgets.FloatSlider,
#         pn.widgets.RangeSlider,
#         pn.widgets.Spinner,
#         pn.widgets.TextInput,
#     ], css_classes=['panel-widget-box'])
#
# row = pn.Row(selector, pn.widgets.ColorPicker(), pn.pane.Str())
#
# def update_value(event):
#     row[2].object = 'Current Value: ' + str(event.new)
#
# def update_widget(event):
#     widget = event.new()
#     widget.param.watch(update_value, 'value')
#     widget.param.trigger('value')
#     row[1] = widget
#
# selector.param.watch(update_widget, 'value')
# selector.param.trigger('value')
#
# pn.Column(
#     '## Dynamically generated user interfaces',
#     row
# )
# pn.template.FastListTemplate(site="Panel", title="Dynamic UI", main=["This example demonstrates **how to dynamically replace widgets in a layout** to create responsive user interfaces", row]).show();

Launching server at http://localhost:49278
